In [1]:
import time
from pathlib import Path
import pandas as pd

In [2]:
data_dir = Path('F:/')/'Research'/'Datasets'/'Wikidata'

In [3]:
nasdaq_tickers = pd.read_csv(data_dir/'NASDAQ_wiki.csv', names=['ticker', 'wikidataid'])
nyse_tickers = pd.read_csv(data_dir/'NYSE_wiki.csv', names=['ticker', 'wikidataid'])

In [4]:
sel_tickers = nyse_tickers

In [5]:
# filename = 'full_company_relationships_triplets_Apr2015_keydata' + '.h5'
# filename = 'full_company_relationships_triplets_keydata_wikidata-20160104-all' + '.h5'
# filename = 'full_company_relationships_triplets_keydata_wikidata-20170102-all' + '.h5'
# filename = 'full_company_relationships_triplets_keydata_wikidata-20180115-all' + '.h5'
# filename = 'full_company_relationships_triplets_keydata_wikidata-20190107-all' + '.h5'
filename = 'full_company_relationships_triplets_keydata_wikidata-20201130-all' + '.h5'
# filename = 'full_company_relationships_triplets_Jun2021_keydata' + '.h5'
company_triplets = pd.read_hdf(data_dir/filename, key='data')

In [6]:
len(company_triplets)

14334

In [7]:
property_df = pd.read_hdf(data_dir/'all_wikidata_properties_keydata.h5', key='data')

In [8]:
company_triplets = pd.merge(company_triplets, property_df, how='left', left_on='relationship', right_on='property')

In [9]:
# filter out 1st order relationships
# company_triplets[(company_triplets.relationship=='P127') & (company_triplets.mkt=='NYSE') & (company_triplets.target.isin(nyse_tickers.wikidataid))].head()

In [10]:
all_relations = pd.read_csv(data_dir/'selected_wiki_connections.csv', sep=' ', names=['relation', 'count'])

In [11]:
len(all_relations)

57

In [12]:
market = 'NYSE'
consolidated_relations = pd.DataFrame()
for i in range(len(all_relations)):
    relation = all_relations.relation[i]
    if len(relation.split('_'))>1:
        df_left = company_triplets[(company_triplets.relationship==relation.split('_')[0]) & (company_triplets.mkt==market)][['source', 'relationship', 'target']]

        df_right = company_triplets[(company_triplets.relationship==relation.split('_')[-1]) & (company_triplets.mkt==market)][['source', 'relationship', 'target']]
        
        composite = pd.merge(df_left, df_right, on='target')[['source_x', 'source_y']]
        composite['relation'] = relation
        composite = composite[composite.source_x != composite.source_y]
        composite.columns = ['source', 'target', 'relation']
        if len(composite) > 0:
            label_left =  company_triplets[(company_triplets.relationship==relation.split('_')[0]) & (company_triplets.mkt==market)].label.values[0]
            label_right =  company_triplets[(company_triplets.relationship==relation.split('_')[-1]) & (company_triplets.mkt==market)].label.values[0]
            # print(f'Doubles - {i}: {label_left}-{label_right}')
            composite['relationname'] = f'{label_left}-{label_right}'
        print(f'Doubles - {i}: {relation}, length {len(composite)}')
        consolidated_relations = pd.concat([consolidated_relations, composite], axis=0)
    else:
        df_left = company_triplets[(company_triplets.relationship==relation.split('_')[0]) & (company_triplets.mkt==market)][['source', 'relationship', 'target']]
        df_right = company_triplets[(company_triplets.relationship==relation.split('_')[0]) & (company_triplets.mkt==market)][['source', 'relationship', 'target']]
        composite = pd.merge(df_left, df_right, on='target')[['source_x', 'source_y']]
        composite['relation'] = relation
        composite = composite[composite.source_x != composite.source_y]
        composite.columns = ['source', 'target', 'relation']
        if len(composite) > 0:
            label =  company_triplets[(company_triplets.relationship==relation.split('_')[0]) & (company_triplets.mkt==market)].label.values[0]
            # print(f'Doubles - {i}: {label_left}-{label_right}')
            composite['relationname'] = f'{label}'
        print(f'Single - {i}: {relation}, length {len(composite)}')
        consolidated_relations = pd.concat([consolidated_relations, composite], axis=0)

consolidated_relations = consolidated_relations.reset_index(drop=True)

Doubles - 0: P361_P361, length 38934
Doubles - 1: P452_P452, length 7978
Doubles - 2: P127_P127, length 2222
Single - 3: P355, length 6
Single - 4: P155, length 4
Doubles - 5: P355_P355, length 6
Doubles - 6: P355_P199, length 0
Doubles - 7: P112_P112, length 10
Doubles - 8: P112_P127, length 1
Doubles - 9: P112_P169, length 0
Doubles - 10: P169_P112, length 0
Doubles - 11: P169_P127, length 0
Doubles - 12: P169_P169, length 0
Doubles - 13: P127_P112, length 1
Doubles - 14: P127_P169, length 0
Doubles - 15: P31_P366, length 0
Doubles - 16: P127_P3320, length 0
Doubles - 17: P463_P463, length 1716
Doubles - 18: P1056_P306, length 0
Doubles - 19: P1056_P1056, length 1158
Doubles - 20: P1056_P452, length 288
Doubles - 21: P31_P452, length 227
Single - 22: P127, length 2222
Single - 23: P156, length 2
Single - 24: P749, length 0
Doubles - 25: P199_P355, length 0
Doubles - 26: P452_P2770, length 0
Doubles - 27: P1344_P1344, length 2
Doubles - 28: P169_P3320, length 3
Doubles - 29: P452_P31,

In [13]:
consolidated_relations.relationname.unique()

array(['part of-part of', 'industry-industry', 'owned by-owned by',
       'subsidiary', 'follows', 'subsidiary-subsidiary',
       'founded by-founded by', 'founded by-owned by',
       'owned by-founded by', 'member of-member of',
       'product or material produced-product or material produced',
       'product or material produced-industry', 'instance of-industry',
       'owned by', 'followed by', 'participant in-participant in',
       'chief executive officer-board member', 'industry-instance of',
       'industry-product or material produced',
       'item operated-item operated',
       'instance of-product or material produced',
       'award received-award received',
       'product or material produced-instance of',
       'board member-chief executive officer',
       'airline alliance-airline alliance',
       'item operated-product or material produced',
       'airline hub-airline hub', 'owned by-parent organization',
       'owned by-subsidiary', 'parent organization-

In [14]:
consolidated_relations.duplicated().sum()

3954

In [15]:
consolidated_relations

,source,target,relation,relationname
0,Q35476,Q212235,P361_P361,part of-part of
1,Q35476,Q483551,P361_P361,part of-part of
2,Q35476,Q533415,P361_P361,part of-part of
3,Q35476,Q675585,P361_P361,part of-part of
4,Q35476,Q905806,P361_P361,part of-part of
...,...,...,...,...
57289,Q66,Q174769,P1056_P121,product or material produced-item operated
57290,Q66,Q291090,P1056_P121,product or material produced-item operated
57291,Q66,Q174769,P1056_P121,product or material produced-item operated
57292,Q66,Q188920,P1056_P121,product or material produced-item operated


In [16]:
ticker_lookup = {}
ticker_inverse_lookup = {}
for i in range(len(sel_tickers)):
    ticker_lookup[sel_tickers.ticker[i]] = sel_tickers.wikidataid[i]
    ticker_inverse_lookup[sel_tickers.wikidataid[i]] = sel_tickers.ticker[i]

In [17]:
consolidated_relations['source_ticker'] = consolidated_relations['source'].map(ticker_inverse_lookup)
consolidated_relations['target_ticker'] = consolidated_relations['target'].map(ticker_inverse_lookup)

In [18]:
consolidated_relations.head(3)

,source,target,relation,relationname,source_ticker,target_ticker
0,Q35476,Q212235,P361_P361,part of-part of,T,AIG
1,Q35476,Q483551,P361_P361,part of-part of,T,WMT
2,Q35476,Q533415,P361_P361,part of-part of,T,BBY


In [19]:
date = 'Nov2020'
mkt = market
filename = 'consolidated_edgelist_' + mkt + '_' + date + '.csv'
print(filename)

consolidated_edgelist_NYSE_Nov2020.csv


In [20]:
consolidated_relations.to_csv(data_dir/filename, index=False)